In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split

df_loaded = pd.read_csv('insurance_data_cleaned.csv')

In [3]:

X_loaded = df_loaded.drop('charges_scaled', axis=1)

y_loaded = df_loaded['charges_scaled'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X_loaded, 
    y_loaded, 
    test_size=0.20, 
    random_state=42 
)

#  Load the Scaler

scaler_y = joblib.load('scaler_y.pkl') 

